In [ ]:
import pandas as pd
import toml
import re
import pygsheets
from tqdm.auto import tqdm
import Bio.Restriction as Restriction
import benchlingapi

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.api as api
from paulssonlab.api.util import base_url
import paulssonlab.cloning.workflow as workflow
import paulssonlab.cloning.util as cloning_util
import paulssonlab.cloning.sequence as sequence
import paulssonlab.cloning.registry as registry
import paulssonlab.cloning.enzyme as enzyme

# Setup

In [ ]:
config = toml.load("config.toml")

In [ ]:
gc = pygsheets.authorize(service_account_file="credentials.json")

In [ ]:
reg = registry.Registry(gc, config["registry"]["folder"])

# PCR simulation test

In [ ]:
seq1 = "TTTT"
seq1a = "TTTTA"
seq1b = "TTTTC"
seq1c = "CCCCCCCCCCCTTTTC"
seq2 = "AAAATTTTAAAATTTTAAAA"
seq3 = "AGTGATTTTTTTCTCCATTCTTTGTGTGTTTTTTTTGTTTTATGAATTTTTTTAACTGATACCCGTTTTTTTGGAAGGAGACCCGTTTTTTTGGAAG"
seq4 = "TTTTTTAAAAAAAGGGGGGGGGGGGGGTTTTTTCCCCCCCCCCCCAAAAAAATTTTTTAAAAAAA"
seq4p = "GGGGGGGGGGGGGG"

In [ ]:
sequence.find_primer_binding_site(seq4p, seq4)

In [ ]:
sequence.slice_seq?

In [ ]:
sequence.find_primer_binding_site(seq5, p2)

In [ ]:
p1 = "TTTTTAGAAGA"
p2 = "TTTTTTAGAGGG"
seq5 = "AGAAGACCCCGGGGCCCTCT"
sequence.pcr(seq5, p1, p2)

In [ ]:
p1 = "ttttttatggacaaag"
p2 = "ttttttcgatacgggg"
seq6 = "atggacaaagactttgatttcgataatattggcaaacgcaccccgtatcg"
sequence.pcr(seq6, p1, p2)

In [ ]:
sequence.find_primer_binding_site(seq6, p2, try_reverse_complement=True)

In [ ]:
sequence.find_primer_binding_site(
    seq6, sequence.reverse_complement(p2), try_reverse_complement=False
)

In [ ]:
sequence.reverse_complement(seq5)

# GG test

In [ ]:
seq1 = get_plib_seq(drive_service, 1)
seq2 = get_plib_seq(drive_service, 82)
seq3 = get_plib_seq(drive_service, 23)
seq4 = get_plib_seq(drive_service, 95)
seq5 = get_plib_seq(drive_service, 110)

In [ ]:
to_join = [
    (sequence.reverse_complement(seq1), Restriction.BsaI, "Name1", "promoter"),
    (sequence.reverse_complement(seq2), Restriction.BsaI, "Name2", "RBS"),
    (seq3, Restriction.BsaI, "Name3", "CDS"),
    (seq4, Restriction.BsaI, "Name4", "terminator"),
    (sequence.reverse_complement(seq5), Restriction.BsaI, "Name5", "misc_feature"),
]

assembly = golden_gate.assemble(to_join, linear=False)
assembly

In [ ]:
reg.get("pLIB1")

In [ ]:
x = reg.get("pLIB1")["_seq"]

In [ ]:
x.annotations

In [ ]:
x.annotations["topology"]

In [ ]:
test_seqs = [reg.get(f"pLIB{plib}")["_seq"] for plib in (1, 82, 23, 95, 110)]

In [ ]:
# TODO: fix linear digests
# TODO: slice circularity

In [ ]:
y = test_seqs[0]

In [ ]:
test_seqs[0].features

In [ ]:
test_digests[0].annotate("foo", overhangs=False).features

In [ ]:
y[0 : len(y)]

In [ ]:
test_digests = [workflow.re_digest_part(seq, Restriction.BsaI) for seq in test_seqs]

In [ ]:
for seq in test_digests:
    print(seq)

In [ ]:
assembly = sequence.assemble(test_digests)
print(assembly)

In [ ]:
print(assembly.circularize())

In [ ]:
with open("/Users/jacob/Downloads/test3.gb", "w") as f:
    f.write(assembly.format("gb"))

# Gibson test

In [ ]:
# gibson two test seqs
# gibson two GG'ed TUs
# check we can PCR UNSes onto JUMP vector
# check we can circularize a real 3G rxn

In [ ]:
to_join = [
    (seq, storage_enzyme),
    (storage_vector_seq, storage_enzyme),
]
plasmid_map = golden_gate.assemble(to_join, linear=False)

# 3G

# Part expressions test

In [ ]:
import tatsu

In [ ]:
# digest
# PCR
# annealed oligos
# dsDNA oligo
# construct part seq from spreadsheet if no implementation

In [ ]:
command = "pLIB225/BsaI,oLIT0.9=oLIT0.10/BsaI"
command = "oLIT9=oLIT10/BsaI"
###
command = "oLIT0.9=oLIT0.10"
command = "oLIT0.9=oLIT0.10/BsaI"
command = "pLIB225<oLIB1,oLIB2>/BsaI"
# command = "oLIT1=oLIT2<oLIB1,oLIB2>/BsaI"
# command = "oLIT1=oLIT2<oLIB1,oLIB2>"
# command = "pLIB225/BsaI"
###
# command = "oLIT0.9=oLIT0.10,oLIT0.9=oLIT0.10"
# command = "oLIT0.9=oLIT0.10/BsaI,oLIT0.9=oLIT0.10/BsaI"
command = "oLIT1=oLIT2<oLIB1,oLIB2>/BsaI,oLIT1=oLIT2<oLIB1,oLIB2>/BsaI"
ast = parser.parse(command)
ast

# Command parsing

In [ ]:
import paulssonlab.cloning.commands as commands

In [ ]:
commands.expr_parser.parse("oLIT1=oLIT2<oLIB1,oLIB2>/BsaI")

In [ ]:
commands.expr_parser.parse(
    "oLIT1=oLIT2<oLIB1,oLIB2>/BsaI,oLIT1=oLIT2<oLIB1,oLIB2>/BsaI"
)

In [ ]:
commands.command_parser.parse(
    "@3(foo, bar, baz, oLIT1=oLIT2<oLIB1,oLIB2>/BsaI, oLIT1=oLIT2<oLIB1,oLIB2>/BsaI)"
)

In [ ]:
command = "@Gib(@GG(UNS1, J23101, BCD11, UNS5), pLIB47<oLIB22,oLIB24>/BsaI)"
ast = commands.command_parser.parse(command)
ast

In [ ]:
# command = (
#     "@3G(UNS1-J23101-BCD11-mVenus-L3S3P11-UNS5, UNS5-J23150-CFP-BCD16-L3S2P55-UNS10)"
# )
# command = "@Gib(@GG(UNS1, J23101, BCD11, UNS5), pLIB47~oLIB22,oLIB24/BsaI)"
# command = "@Gib(@GG(UNS1, J23101, BCD11, UNS5), @PCR(pLIB47, oLIB22, oLIB24)/BsaI)"
# command = (
#     "@Gib(@GG(UNS1, J23101, BCD11, UNS5), @RE(@PCR(pLIB47, oLIB22, oLIB24), BsaI))"
# )

In [ ]:
part_sheet.sync?

In [ ]:
def get_named_sequence(name, part_sheet, plasmid_maps):
    # try plasmid, strain, part
    pass


def goldengate(*args):
    return "gg", args


def threeg(*args):
    return "3g", args


commands = {"GG": goldengate, "3G": threeg}

get_named_sequence("pLIB27")

In [ ]:
reference_parser = tatsu.compile(grammar_preamble + reference_grammar)


class CloningCommandSemantics(object):
    def __init__(self, commands):
        self.commands = commands

    def command(self, ast):
        if ast.command_name not in self.commands:
            raise tatsu.semantics.SemanticError(
                "command must be one of: {}".format(
                    ", ".join([f"@{k}" for k in commands.keys()])
                )
            )
        command = self.commands[ast.command_name]
        return command(ast.arguments)

    def int_(self, ast):
        return int(s)

    def float_(self, ast):
        return float(s)

    def name(self, ast):
        return ast.name

    def assembly(self, ast):
        return ast.assembly


# parser.parse(command, semantics=CloningCommandSemantics())

In [ ]:
# GG
command = "@GG(J23101, BCD11, mVenus, L3S3P11, p121/BsaI)"
# 3G
# command = "@3G(@GG(UNS1, J23101, BCD11, mVenus, L3S3P11, UNS5), @GG(UNS5, J23150, CFP, BCD16, L3S2P55, UNS10), JUMP_p15a_UNS1_UNS10)"

In [ ]:
parser.parse(command, semantics=CloningCommandSemantics(commands))